# ОТЧЕТ

#### 1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

Данные столбцы имеют пропуски:    
first_blood_time:   0.20110048338990025      
first_blood_team:   0.20110048338990025     
first_blood_player1:   0.20110048338990025     
first_blood_player2:   0.4524015221639412     
radiant_bottle_time:   0.16138023243854777     
radiant_courier_time:   0.007117144914121156     
radiant_flying_courier_time:   0.2826185333744729     
radiant_first_ward_time:   0.018883060783708733     
dire_bottle_time:   0.1660290033940142     
dire_courier_time:   0.00695258665021084     
dire_flying_courier_time:   0.2684150982207138     
dire_first_ward_time:   0.018780211868764784   

1) в 28% случаев команда не покупает "flying_courier" в первые 5 минут игры     
2) почти в 2% случаев, команда Radiant не ставит "Ward" в первые 5 минут игры

#### 2. Как называется столбец, содержащий целевую переменную? ####
Ответ: столбец 'radiant_win'содержит целевую переменную

#### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями?  Какое качество при этом получилось? ####

1) Кросс-валидация для градиентного бустинга с 30 деревьями длится 31.34 секунды    
2) Качество составило 0.69

#### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев? ####

1) Да, имеет. Показатель метрики качества со 100 (0.71) и 150 (0.72) деревьями больше, чем с 30 (0.69).    
2) Для ускорения можно уменьшить глубину деревьев 

# Подход 1: градиентный бустинг "в лоб"


In [17]:
import pandas as pd
from pprint import pprint 
import time
import datetime
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier



### 1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [4]:
featuresTrain = pd.read_csv('data/features.csv', index_col='match_id')
featuresTrain = featuresTrain.drop([
		'duration', 
		'tower_status_radiant', 
		'tower_status_dire', 
		'barracks_status_radiant', 
		'barracks_status_dire'
		], 
		axis=1)

### 2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [5]:
def defineEmptyColumn(df):
	# count - пременная, в которой хранится кол-во пустых ячеек для каждого столбца
	count = 0
	# rowNum - переменная, в которой хранится кол-во строк в DataFrame
	rowNum = len(df.index)
	# Перебор всех столбцов
	for colName in df:
		count = df[colName].isnull().sum()
		if count > 0:
			print(colName + ":   " + repr(count/rowNum))

In [6]:
defineEmptyColumn(featuresTrain)

first_blood_time:   0.20110048338990025
first_blood_team:   0.20110048338990025
first_blood_player1:   0.20110048338990025
first_blood_player2:   0.4524015221639412
radiant_bottle_time:   0.16138023243854777
radiant_courier_time:   0.007117144914121156
radiant_flying_courier_time:   0.2826185333744729
radiant_first_ward_time:   0.018883060783708733
dire_bottle_time:   0.1660290033940142
dire_courier_time:   0.00695258665021084
dire_flying_courier_time:   0.2684150982207138
dire_first_ward_time:   0.018780211868764784


In [8]:
# 1) в 28% случаев команда не покупает "flying_courier" в первые 5 минут игры
# 2) почти в 2% случаев, команда Radiant не ставит "Ward" в первые 5 минут игры

### 3. Замените пропуски на нули с помощью функции fillna().

In [9]:
def cleanEmptyColumn(df):
	df = df.fillna(0)
	return df

### 4) Какой столбец содержит целевую переменную? Запишите его название.

Ответ: столбец 'radiant_win'содержит целевую переменную

### 5. Обучить градиентный бустинг 

In [ ]:
def gradientBoosting(trainData):
	XTrain = trainData.drop('radiant_win',1)
	yTrain = trainData['radiant_win'].to_frame()

	kf = KFold(yTrain.size, n_folds=5, shuffle=True, random_state=241)
	numTrees = [10, 20, 30, 100, 150]
	for num in numTrees:
		print("Количество деревьев: ", num)
		model = GradientBoostingClassifier(n_estimators=num, random_state=241)
		startTime = datetime.datetime.now()
		scores = cross_val_score(model, XTrain, yTrain.values.ravel(), cv=kf, scoring='roc_auc', n_jobs=-1)
		print('Время обучения: ', datetime.datetime.now() - startTime)
		print(scores)
		print("_____")

In [15]:
# Замена пропусков на нули
trainData = cleanEmptyColumn(featuresTrain)
# Функция обучения градинтного бустинга
gradientBoosting(trainData)

Количество деревьев:  10
Время обучения:  0:00:11.825738
[0.66943496 0.65627754 0.66390454 0.66281223 0.66950933]
_____
Количество деревьев:  20
Время обучения:  0:00:24.167875
[0.68939011 0.67567277 0.68201026 0.67977584 0.68741889]
_____
Количество деревьев:  30
Время обучения:  0:00:31.338620
[0.69641668 0.68365441 0.68731862 0.68728458 0.69280674]
_____
Количество деревьев:  100
Время обучения:  0:01:46.529449
[0.71211028 0.70275727 0.70340522 0.70450889 0.70884943]
_____
Количество деревьев:  150
Время обучения:  0:02:33.775093
[0.71546705 0.70731614 0.70793497 0.70887821 0.71252658]
_____
